<a href="https://colab.research.google.com/github/ACRainieri/pyspark/blob/main/Projeto_WEB_DSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install Flask


In [ ]:
from flask import Flask

app = Flask(__name__)

@app.route("/")
def index():
  return "Index!"

@app.route("/hello")
def hello():
  return "Hello World!!"

@app.route("/members")
def members():
  return "Members"

@app.route("/members/<string:name>/")  
def getMember(name):
  return name

if __name__ == "__main__":
  app.run(host='127.0.0.1', port=5000)

Web Scrapping

In [ ]:
import urllib.request

with urllib.request.urlopen("https://www.cifraclub.com.br/van-halen/cant-stop-lovin-you/") as url:
  page = url.read()

from bs4 import BeautifulSoup

soup  = BeautifulSoup(page, "html.parser")

print(soup.title.string)
print(soup)

#print(soup.string_container)
#print(soup.find_all('a'))

#tables = soup.find('tables')


Importando dados de pagina Web

In [ ]:
import pandas as pd
import requests 
from bs4 import BeautifulSoup
from tabulate import tabulate

res = requests.get("https://www.nationmaster.com/country-info/stats/Media/Internet-users")

soup = BeautifulSoup(res.content, 'lxml')

#Extrair a tabela html
table = soup.find_all('table')[0]

#Converter a tabela HTML em um framework pandas
df = pd.read_html(str(table))

print(df)


#Convert para formato Json
print(df[0].to_json(orient='records'))

print('\nTabela formatada por pandas/tabulate:\n')
print(tabulate(df[0], headers='keys', tablefmt='psql'))